# Let's build an agentic search loop!

This notebook has a basic agentic search loop

* We have a set of furniture in our catalog
* We tell the Agent our preferences
* The agent uses the search tool to recommend furniture

In this notebook we mostly get a feel for how the overall loop works by unrolling it step by step

In [ ]:
!pip install git+https://github.com/softwaredoug/cheat-at-search.git
from cheat_at_search.data_dir import mount
mount(use_gdrive=True)    # colab, share data across notebook runs on gdrive
# mount(use_gdrive=False) # <- colab without gdrive
# mount(use_gdrive=False, manual_path="/path/to/directory")  # <- force data path to specific directory, ie you're running locally.


  Cloning https://github.com/softwaredoug/cheat-at-search.git to /tmp/pip-req-build-snltpyje
  Running command git clone --filter=blob:none --quiet https://github.com/softwaredoug/cheat-at-search.git /tmp/pip-req-build-snltpyje
  Resolved https://github.com/softwaredoug/cheat-at-search.git to commit 0a40c55184b360876011926ca9836707840a5277
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.3/745.3 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 8.2 MB/s eta 0:00:00
  Created wheel for cheat_at_search: filename=cheat_at_search-0.1.0-py3-none-any.whl size=63360 sha256=ab0debd8cee3df5641f235562162ca718715c576f7a479fd365b763a4ef6efcd
  Stored in directory: /tmp/pip-ephem-wheel-cache-rurypib1/wheels/ea/96/d3/e69453e5f86e6f891864e5b5baf2b11ffda2d0981d

## Get an OpenAI Key

This will prompt you for an OpenAI Key to interact with GPT-5

In [ ]:
from cheat_at_search.data_dir import key_for_provider
from openai import OpenAI

OPENAI_KEY = key_for_provider("openai")

openai = OpenAI(api_key=OPENAI_KEY)

## Load the Wayfair corpus

We'll recommend products only from this corpus

In [ ]:
from cheat_at_search.wands_data import corpus
corpus

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,doc_id,title,description,category,sub_category,cat_subcat
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0,"[overallwidth-sidetoside:64.7, dsprimaryproduc...",0,solid wood platform bed,"good , deep sleep can be quite difficult to ha...",Furniture,Bedroom Furniture,Furniture / Bedroom Furniture
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0,"[capacityquarts:7, producttype : slow cooker, ...",1,all-clad 7 qt . slow cooker,"create delicious slow-cooked meals , from tend...",Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0,"[features : keep warm setting, capacityquarts:...",2,all-clad electrics 6.5 qt . slow cooker,prepare home-cooked meals on any schedule with...,Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0,"[overallwidth-sidetoside:3.5, warrantylength :...",3,all-clad all professional tools pizza cutter,this original stainless tool was designed to c...,Browse By Brand,All-Clad,Browse By Brand / All-Clad
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0,"[compatibledoorthickness:1.375 '' , countryofo...",4,baldwin prestige alcott passage knob with roun...,the hardware has a rich heritage of delivering...,Home Improvement,Doors & Door Hardware,Home Improvement / Doors & Door Hardware
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42989,42989,malibu pressure balanced diverter fixed shower...,Shower Panels,Home Improvement / Bathroom Remodel & Bathroom...,the malibu pressure balanced diverter fixed sh...,producttype : shower panel|spraypattern : rain...,3.0,4.5,2.0,"[producttype : shower panel, spraypattern : ra...",42989,malibu pressure balanced diverter fixed shower...,the malibu pressure balanced diverter fixed sh...,Home Improvement,Bathroom Remodel & Bathroom Fixtures,Home Improvement / Bathroom Remodel & Bathro...
42990,42990,emmeline 5 piece breakfast dining set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,,basematerialdetails : steel| : gray wood|ofhar...,1314.0,4.5,864.0,"[basematerialdetails : steel, : gray wood, of...",42990,emmeline 5 piece breakfast dining set,,Furniture,Kitchen & Dining Furniture,Furniture / Kitchen & Dining Furniture
42991,42991,maloney 3 piece pub table set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,this pub table set includes 1 counter height t...,additionaltoolsrequirednotincluded : power dri...,49.0,4.0,41.0,[additionaltoolsrequirednotincluded : power dr...,42991,maloney 3 piece pub table set,this pub table set includes 1 counter height t...,Furniture,Kitchen & Dining Furniture,Furniture / Kitchen & Dining Furniture
42992,42992,fletcher 27.5 '' wide polyester armchair,Teen Lounge Furniture|Accent Chairs,Furniture / Living Room Furniture / Chairs & S...,"bring iconic , modern style to your space in a...",legmaterialdetails : rubberwood|backheight-sea...,1746.0,4.5,1226.0,"[legmaterial

### Index the furniture

We'll index title and description with basic stemming to be able to retrieve them

In [ ]:
from searcharray import SearchArray
from cheat_at_search.tokenizers import snowball_tokenizer

corpus['title_snowball'] = SearchArray.index(corpus['title'].fillna(''), snowball_tokenizer)
corpus['description_snowball'] = SearchArray.index(corpus['description'].fillna(''), snowball_tokenizer)

2025-12-02 02:32:54,066 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2025-12-02 02:32:54,074 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2025-12-02 02:32:54,077 - searcharray.indexing - INFO - Tokenizing 42994 documents


...

INFO:searcharray.indexing:Batch tokenization complete


2025-12-02 02:33:03,676 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2025-12-02 02:33:03,861 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete


## Create a furniture products search function

Here is a function that searches a Wayfair product dataset. It's just a Python function that returns top 10 pieces of furniture.

Right now we'll call it directly, soon we'll help ChatGPT interact with this.

In [ ]:
import numpy as np
from typing import Union

def search_furniture(keywords: str) -> list[dict[str, Union[str, int, float]]]:
    """Search the available furniture products, get top 10 furniture.

    This is just a naive BM25 / keyword search of the product title and description.
    Don't expect sophisticated synonyms or semantic search. Just basic keyword with
    some stemming.

    """
    print("search", keywords)
    required_keywords = [term[1:] for term in keywords.split() if term.startswith("+")]
    bm25_scores = np.zeros(len(corpus))
    for term in snowball_tokenizer(keywords):
        bm25_scores += corpus['title_snowball'].array.score(term) * 9.3
        bm25_scores += corpus['description_snowball'].array.score(term) * 4.1

    for required_term in snowball_tokenizer(" ".join(required_keywords)):
        required_score = (corpus['title_snowball'].array.score(required_term) +
                          corpus['description_snowball'].array.score(required_term))
        bm25_scores[required_score == 0] = 0

    top_k_indices = np.argsort(bm25_scores)[-10:][::-1]
    bm25_scores = bm25_scores[top_k_indices]
    top_movies = corpus.iloc[top_k_indices].copy()
    top_movies.loc[:, 'score'] = bm25_scores

    results = []
    for id, row in top_movies.iterrows():
        results.append({
            'id': row['doc_id'],
            'title': row['title'],
            'description': row['description'],
            'score': row['score']
        })
    return results



results = search_furniture("gray leather cocktail table")

search gray leather cocktail table


## Add a tool to evaluate

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, Literal
from cheat_at_search.wands_data import judgments

class SearchResult(BaseModel):
    """A furniture search result."""
    doc_id: int = Field(..., description="The product id")
    title: str = Field(..., description="The title of the product (from the corpus)")
    description: str = Field(..., description="The description of (from the corpus)")
    explanation: str = Field(..., description="Why this is relevant for the search in your words")

class SearchResults(BaseModel):
    """Ranked search results in order of relevance."""
    search_results: list[SearchResult] = Field(..., description="The search results")

class ScoredSearchResult(BaseModel):
    """A search result with a score."""
    result: SearchResult = Field(..., description="The search result")
    user_satisfaction: Optional[Literal['😁', '🫤', '😥']] = Field(..., description="How happy this result will make the user")

class ScoredSearchResults(BaseModel):
    """Ranked search results in order of relevance."""
    search_results: list[ScoredSearchResult] = Field(..., description="The search results")
    message: Optional[str] = Field(..., description="A message to the user about the results")
    tool_reflection: Optional[str] = Field(...,
                                           description="What would you tell the next iteration about using the search tool (not specific to this query, just in general)? Format in a way that could be inserted into a prompt")
    tool_queries_satisfying_user: list[str] = Field(...,description="The list of queries you issued to the underlying search tool that yield satisfing results for the user")

def score(query: str, results: SearchResults) -> ScoredSearchResults:
    """Return whether the results for the query will satisfy the user

    * If you call 'score' on a result, and it doesn't have a user_satisfaction label, its unknown whether the user would like it
    * If you get a '😥' then the user doesn't like it
    * If you get a '😁' then the user likes it
    * If you get a '🫤' then the user is meh
    """
    print("score", query, len(results.search_results))
    scored_results = []
    available_queries = judgments['query'].unique()
    skip_score = False
    if query not in available_queries:
        skip_score = True
    scores = judgments[judgments['query'] == query]
    max_score = 2
    for result in results.search_results:
        score = 0
        label = '😥'
        available_doc_ids = scores['doc_id'].unique()
        if result.doc_id in available_doc_ids:
            score = scores[scores['doc_id'] == result.doc_id]['grade'].values[0]
            if score == 2:
                label = '😁'
            elif score == 1:
                label = '🫤'
            elif score == 0:
                label = '😥'
        scored_results.append(ScoredSearchResult(
            result=result,
            user_satisfaction=label if not skip_score else None
        ))
    return ScoredSearchResults(search_results=scored_results,
                               message="",
                               tool_queries_satisfying_user=[],
                               tool_reflection="")

In [ ]:
search_results = []

for result in results:
    search_results.append(SearchResult(
        doc_id=result['id'],
        title=result['title'],
        description=result['description'],
        explanation=str(result['score'])
    ))

search_results = SearchResults(search_results=search_results)
score("gray leather cocktail table", search_results)

score gray leather cocktail table 10


ScoredSearchResults(search_results=[ScoredSearchResult(result=SearchResult(doc_id=24399, title='radersburg leather cocktail table', description="whether you 're serving up trays of tasty crudites at your next elegant gathering or simply looking for a stylish finishing touch to your living room seating ensemble , this chic cocktail table gets the job done . a medium cherry finish pairs with leather-inspired inserts for a touch of rustic charm , while a steel frame and lower storage shelf adds a dash of sleek style and functionality to your space .", explanation='84.16834878921509'), user_satisfaction='🫤'), ScoredSearchResult(result=SearchResult(doc_id=10350, title='boston cocktail table', description='product highlights a round weathered grey casual cocktail table with plank-style details , double “ x ” legs , and an “ x ” base . product information collection : bridgeport dimension : height in : 20 dimension : length in : 36.13 dimension : width in : 36.13', explanation='63.67524099349

## Describe the search tool to the LLM

There is a specific schema for telling OpenAI about our tools / functions. However, the cheat at search library has added some conveniences:

* We use the function name as the name to OpenAI
* We use the doc string to get a description
* The typing information gets encoded in parameters and return value

So IMPORTANTLY -- all these things are part of the prompt

### Annoying serialization / deserialization

When we get it in an OpenAI-friendly format, we also keep around some book-keeping for annoying serialization / deserialization of the arguments

With this we get some plumbing information in a 3-tuple
* The arguments to pass (as one pydantic struct)
* The tool as OpenAI sees it
* The function to call to delegate to this tool

Don't get too lost in the weeds here. In future notebooks, cheat-at-search helper code will just do this for you behind the scenes.

In [ ]:
from cheat_at_search.agent.pydantize import make_tool_adapter
search_tool = make_tool_adapter(search_furniture)
score_tool = make_tool_adapter(score)

tool_info = {search_furniture.__name__: search_tool,
             score.__name__: score_tool}
tool_info

{'search_furniture': (cheat_at_search.agent.pydantize.Search_furnitureArgs,
  {'type': 'function',
   'name': 'search_furniture',
   'description': "Search the available furniture products, get top 10 furniture.\n\n    This is just a naive BM25 / keyword search of the product title and description.\n    Don't expect sophisticated synonyms or semantic search. Just basic keyword with\n    some stemming.",
   'parameters': {'properties': {'keywords': {'title': 'Keywords',
      'type': 'string'}},
    'required': ['keywords'],
    'title': 'Search_furnitureArgs',
    'type': 'object'}},
  <function cheat_at_search.agent.pydantize.make_tool_adapter.<locals>.call_from_tool(d: dict)>),
 'score': (cheat_at_search.agent.pydantize.ScoreArgs,
  {'type': 'function',
   'name': 'score',
   'description': "Return whether the results for the query will satisfy the user\n\n    * If you call 'score' on a result, and it doesn't have a user_satisfaction label, its unknown whether the user would like it 

## Issue the requested calls to your search tool

Now we do the magic of calling the tools directly

You can ignore the `annoying_tool_marshalling` its doing some lookups and plumbings to go between the JSON arguments and the Python world we have here.

The important thing is that we recieve a tool call request, we call the requested tool (here by doing a lookup and getting `tool_fn` that just wraps the search function)

Then we go on to append those all back into th inputs, with a JSON response, the call id, and a note to OpenAI this is a "function_call_output"

In [ ]:
def annoying_tool_marshalling(item) -> dict:

    # Lookup how the agent wants to call the tool
    tool_name = item.name
    tool = tool_info[tool_name]
    ToolArgsModel = tool[0]
    tool_fn = tool[2]
    fn_args: ToolArgsModel = ToolArgsModel.model_validate_json(item.arguments)

    # The tool call function itself (ie search)
    # wrapped in something helping with serialization
    print(f"Calling {tool_name} with {fn_args}")
    py_resp, json_resp = tool_fn(fn_args)

    # 4. Provide function call results to the model
    return {
        "type": "function_call_output",
        "call_id": item.call_id,
        "output": json_resp,
    }



## Put it all in one loop

In [ ]:
import textwrap
from pydantic import BaseModel, Field

system_prompt = """
Users are searching a catalog of furniture, and you're going to help them find
results that satisfies them!

* Use the search tool (search_furniture) to retrieve search results (notice its limitations)
* Use the scoring tool (score) to see if this will satisfy them

We're a poor, small company, so our search is not good. It's just a keyword search. We need you,
tireless agent, to use trial and error with our search tool!

Use trial and error with the search tool to find items that satisfy users. Try to find 10 results.

If you only find results that will actively dissatisfy the user, just omit those results. Even 0 results!

Any calls to search_furniture that yield results that seem to satisfy users, put them in the tool_queries_satisfying_user list.

"""
# Finally, for tool_reflection, give generic, broadly applicable guidance regardless of the query. We need to take that and use it for any query.

def agentic_search(query: str, hint=None, summary=True) -> str:

    inputs = []
    inputs.append({"role": "system", "content": system_prompt})

    if hint:
        inputs.append({"role": "user", "content": "Here is a hint: " + hint})

    inputs.append({"role": "user", "content": "The users query:" + query})


    tool_calls = True
    resp = None
    while tool_calls:
        print("Calling OpenAI")
        resp = openai.responses.parse(
            model="gpt-5",
            input=inputs,
            tools=[tool[1] for tool in tool_info.values()],
            reasoning={
                "effort": "medium",
                "summary": "auto" if summary else "none"
            },
            text_format=ScoredSearchResults
        )
        print("...done")
        inputs += resp.output
        if summary:
            print("\n## Reasonings: ")
            for item in resp.output:
                if item.type == "reasoning":
                    for summary_item in item.summary:
                        print(textwrap.fill(summary_item.text, 80), "\n")
                    item.summary = []

        for item in resp.output:
            tool_calls = False
            if item.type == "function_call":
                tool_calls = True
                # *** Get the tool, and package
                # up the call to the tool (our python function)
                tool_response = annoying_tool_marshalling(item)

                # 4. Provide function call results to the model
                inputs.append(tool_response)
    return resp.output_parsed


results = agentic_search('small ladies rocker swivel recliner')

Calling OpenAI
...done

## Reasonings: 
**Searching for furniture options**  I need to use tools since the user is
looking for a furniture catalog. I'll run a search for furniture using different
keyword combinations. Since we're using a simple BM25 keyword search, I might
need to rely on a bit of trial and error to find 10 results that meet the user's
needs. I can use the scoring tool to check if those results satisfy the user,
based on different satisfaction labels I’ll receive. 

**Searching for various recliners**  I need to run multiple search calls with
different keywords like "small rocker swivel recliner," "ladies rocker
recliner," and others. Since the catalog is generic, I won’t know what’s
available until I start the search. After that, I’ll use the scoring tool to
check overall satisfaction for the retrieved results. I’m looking for items
rated with '😁' as satisfying and might consider including those rated '🫤', but I
want to focus on satisfying the user’s needs. 

**Priori

In [ ]:
print("msg", textwrap.fill(results.message, 80))
hint = results.tool_reflection
print("reflection", textwrap.fill(results.tool_reflection, 80))
for result in results.search_results:
    print(f"{result.user_satisfaction} -- {result.result.title} ({result.result.doc_id})")

msg I tried many keyword combinations. The catalog didn’t surface items explicitly
labeled “small/petite ladies” with both swivel and rocker, but these two swivel
glider recliners are close matches (rocking/gliding + swivel + recline) and are
typically more compact than full-size recliners. If you can share a target seat
width or max overall width, I can refine further and try again.
reflection - Issue multiple keyword variants and include likely synonyms (e.g., swivel
rocker recliner, swivel glider recliner, compact/petite/small recliner, nursery
glider). - Include dimensional cues (e.g., 25" wide, 28" wide, compact) to
surface smaller-scale pieces. - The search is pure keyword: try singular/plural
and re-order terms; avoid relying on semantics. - Always run the scoring tool
and filter out results with negative satisfaction; keep neutral ones if needed.
- Favor results that explicitly list all requested features in the
title/description; dimensions help when size is required.
🫤 -- arl

In [ ]:
results.tool_queries_satisfying_user

[]

In [ ]:
results = agentic_search('small ladies rocker swivel recliner',
                         hint=hint)

Calling OpenAI
...done

## Reasonings: 
**Searching for furniture options**  I need to act as a search helper using the
available tools. The user has given clear instructions: I'll include multiple
keyword variants and synonyms while ensuring to incorporate dimensional cues to
fine-tune the search for smaller-scaled items. It’s essential to run the scoring
tool to filter out negative results and prioritize options that explicitly list
all requested features. I’ll repeatedly use the search_furniture tool to gather
top results that match the user’s query for a "small ladies rocker swivel
recliner." I'll keep experimenting until I get satisfying results. 

**Using scoring tool for results**  The scoring tool will provide satisfaction
labels for each result, marked as unknown, positive, neutral, or negative. My
goal is to gather the top results that are either positive or neutral, filtering
out any negative ones. I may need to run multiple searches and score the results
each time to evalua

In [ ]:
print("msg", textwrap.fill(results.message, 80))
hint = results.tool_reflection
print("reflection", textwrap.fill(results.tool_reflection, 80))
for result in results.search_results:
    print(f"{result.user_satisfaction} -- {result.result.title} ({result.result.doc_id})")

msg I prioritized compact swivel glider/rocker recliners aimed at women/nursery use.
These five options are the closest matches I could surface; several explicitly
include swivel + glider/rock and recline, and one lists a compact width (30.5").
If you have a target max width (e.g., ≤28"), I can refine to only those, or
focus strictly on models that explicitly say both “swivel” and “rocker/glider.”
reflection General guidance for this catalog: use multiple keyword variants with explicit
features and sizes. Include synonyms and re-order terms (swivel rocker recliner,
swivel glider recliner, rocking glider recliner). Add numeric width cues (e.g.,
25", 26", 28") and descriptors like compact, petite, small, apartment size. Try
singular/plural forms. Avoid relying on semantic inference—spell out every
desired feature in the query. Always run the scoring tool after each search,
then filter out results with negative satisfaction and keep neutral ones if
positives aren’t found. Favor listings t

In [ ]:
from cheat_at_search.wands_data import judgments

judgments[judgments['query'] == 'small ladies rocker swivel recliner']

,query_id,query,query_class,doc_id,id,product_id,label,grade
175412,400,small ladies rocker swivel recliner,Recliners,2266,41010,2266,Partial,1.0
175413,400,small ladies rocker swivel recliner,Recliners,2517,157487,2517,Partial,1.0
175414,400,small ladies rocker swivel recliner,Recliners,2636,157479,2636,Partial,1.0
175415,400,small ladies rocker swivel recliner,Recliners,9210,157489,9210,Partial,1.0
175416,400,small ladies rocker swivel recliner,Recliners,9420,41013,9420,Partial,1.0
175417,400,small ladies rocker swivel recliner,Recliners,9492,157476,9492,Partial,1.0
175418,400,small ladies rocker swivel recliner,Recliners,11115,157480,11115,Partial,1.0
175419,400,small ladies rocker swivel recliner,Recliners,11230,157483,11230,Partial,1.0
175420,400,small ladies rocker swivel recliner,Recliners,14623,157493,14623,Partial,1.0
175421,400,small ladies rocker swivel recliner,Recliners,17222,41011,17222,Partial,1.0
